In [1]:
## TESTS START FROM LINE 2156 (tomohara-aliases.bash)

## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-py-commands

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

## TAKES EXTREMELY LONG TIME TO SOURCE THE MAIN BASH FILEII
# source tomohara-aliases.bash

alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3443)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3443

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-py-commands/test-3443


In [6]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [7]:
export PYTHON_CMD="/usr/bin/time python -u"
export PYTHON="$NICE $PYTHON_CMD"
export PYTHONPATH="$HOME/python:$PYTHONPATH"

In [8]:
# add-python-path(pkg-dir): add PKG-DIR to PATH and PARENT to PYTHONPATH
## HACK: make sure Mezcla/mezcla resolves before python/mezcla
## OLD: export PYTHONPATH="$HOME/python/Mezcla:$PYTHONPATH"
function add-python-path () {
    local package_path="$1"
    local parent_path=$(realpath "$package_path/..")
    # add package to path (e.g., $HOME/python/Mezcla/mezcla)
    prepend-path "$package_path"
    # add parent to python-path spec (e.g., $HOME/python/Mezcla)
    export PYTHONPATH="$parent_path:$PYTHONPATH"
}
# EX: mezcla-devel; which system.py | grep -i Mezcla-main => ""
# note: mezcla-devel should be stable version of mezcla-tom
## OLD: alias mezcla-devel='add-python-path $HOME/programs/python/mezcla-tom/mezcla'
alias mezcla-devel='add-python-path $HOME/python/Mezcla/mezcla'
alias mezcla-main='add-python-path $HOME/python/Mezcla-main/mezcla'
alias mezcla-tom='add-python-path $HOME/python/Mezcla-tom/mezcla'
# Add mezcla-devel unless another version in path
if [[ ! "$PATH" =~ mezcla ]]; then
    ## OLD: mezcla-devel
    ## TODO: echo "Warning: mezcla not in PATH"
    true
fi

In [22]:
## FUNCTIONS REQUIRED FOR SMOOTH RUNNING OF THE TESTS
function ps-all () { 
    local pattern="$1";
    local pager=cat;
    if [ "$pattern" = "" ]; then 
        pattern="."; 
        pager=$PAGER
    fi;
    ps_mine.sh --all | $EGREP -i "((^USER)|($pattern))" | $pager;
    }
# Path settings
# TODO: define a function for removing duplicates from the PATH while
# preserving the order
function show-path-dir () { (echo "${1}:"; printenv "$1" | perl -pe "s/:/\n/g;") | $PAGER; }
alias show-path='show-path-dir PATH'
# append-path(path): appends PATH to environment variable unless already there
## TODO: function in-path { local path=$(tr ":" "\n" | $GREP "^$1$$"); return ($path != ""); }
# TODO: add force argument to ensure last (or first)
function append-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="${PATH}:$1"; fi }
function prepend-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="$1:${PATH}"; fi }
# TODO: rework append-/prepend-path and python variants via generic helper
function append-python-path () { export PYTHONPATH=${PYTHONPATH}:"$1"; }
function prepend-python-path () { export PYTHONPATH="$1":${PYTHONPATH}; }
 

In [28]:
# add-python-path $temp_dir
add-python-path $temp_dir

In [29]:
alias ps-python-full='ps-all python'
# note: excludes ipython and known system-related python scripts
alias ps-python='ps-python-full | $EGREP -iv "(screenlet|ipython|egrep|update-manager|software-properties|networkd-dispatcher)"'
alias show-python-path='show-path-dir PYTHONPATH'

In [30]:
## ERROR 1 - process ID list syntax error

# $ ps-python-full
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).


error: process ID list syntax error

Usage:
 ps [options]

 Try 'ps --help <simple|list|output|threads|misc|all>'
  or 'ps --help <s|l|o|t|m|a>'
 for additional help text.

For more details see ps(1).


: 1

In [33]:
## ERROR 2 - bash : command not found and process ID list syntax error
# $ ps-python
# | bash: -iv: command not found
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).


bash: $: command not found
bash: syntax error near unexpected token `}'
bash: syntax error near unexpected token `|'
bash: syntax error near unexpected token `|'
bash: syntax error near unexpected token `|'
bash: syntax error near unexpected token `|'
bash: syntax error near unexpected token `|'
bash: Try: command not found
bash: or: command not found
bash: syntax error near unexpected token `help'
bash: syntax error near unexpected token `('


: 2

In [32]:
## show-python-path DISPLAYS ALL THE AVAILABLE PATHS OF PYTHON
show-python-path

PYTHONPATH:
/tmp/test-py-commands
/tmp/test-py-commands
/tmp/test-py-commands
/tmp/test-py-commands
/home/aveey/python



In [34]:
# Python-lint filtering
# python-lint-full(filename): complete output from pylint, with caret-based
# context indicators retained by substiting carriage return for newline.
# python-lint-work(filename): pylint with moderate filtering
# python-lint(filename): pylint with usual filtering
# TODO: specify exclusion types in pylint command line (e.g., invalid-name)
# example: "C:674, 0: Invalid constant name "term_freq" (invalid-name)"
# example: "run_ner.py:413:0: C0330: Wrong continued indentation (add 8 spaces).
#                   'B-DATE', 'I-DATE', 'B-DOCTOR', 'I-DOCTOR', 'B-LOCATION', 'I-LOCATION', 'B-AGE', 'I-AGE',
#                   ^       | (bad-continuation)
# TODO: make Mercurial root-to-python-path hack optional
# TODO: add similar aliases for pep8 and pyflakes
# TODO: handle continutations of statements without indentation:
#   ner_eval/ner_detokenize.py:11:45: C0326: Exactly one space required after comma
#   parser.add_argument('--output_file', type=str,  help='')
#                                      ^ (bad-whitespace)

function python-lint-full() { 
    local root=$(hg root 2> /dev/null);
    ## TODO: --persistent=n (to avoid caching)
    PYTHONPATH="$root:.:$PYTHONPATH" $NICE pylint "$@" | perl -00 -ne 'while (/(\n\S+:\s*\d+[^\n]+)\n( +)/) { s/(\n\S+:\s*\d+[^\n]+)\n( +)/$1\r$2/mg; } print("$_");' 2>&1 | $PAGER;
}

In [36]:
python-lint-full 
# NEED TO WORK ON THIS - CALL IT A DAY (2022-07-23 23:10 +05:45 GMT) 

usage: pylint [options]

options:
  -h, --help            show this help message and exit

Commands:
  Options which are actually commands. Options in this group are mutually exclusive.

  --rcfile RCFILE       Specify a configuration file to load.
  --output OUTPUT       Specify an output file.
  --help-msg HELP_MSG [HELP_MSG ...]
                        Display a help message for the given message id and
                        exit. The value may be a comma separated list of
                        message ids.
  --list-msgs           Display a list of all pylint's messages divided by
                        whether they are emittable with the given interpreter.
  --list-msgs-enabled   Display a list of what messages are enabled, disabled
                        and non-emittable with the given configuration.
  --list-groups         List pylint's message groups.
  --list-conf-levels    Generate pylint's confidence levels.
  --list-extensions     List available extensions.
  --full-d

                        and 'info' which contain the number of messages in
                        each category, as well as 'statement' which is the
                        total number of statements analyzed. This score is
                        used by the global evaluation report (RP0004).
                        (default: max(0, 0 if fatal else 10.0 - ((float(5 *
                        error + warning + refactor + convention) / statement)
                        * 10)))
  --score <y or n>, -s <y or n>
                        Activate the evaluation score. (default: True)
  --msg-template <template>
                        Template used to display messages. This is a python
                        new-style format string used to format the message
                        information. See doc for all details. (default: )

Messages control:
  Options controlling analysis messages

  --confidence <levels>
                        Only show warnings with the listed confidence levels.


                        decorated function. (default: [])

Refactoring:
  Looks for code which can be refactored.

  --max-nested-blocks <int>
                        Maximum number of nested blocks for function / method
                        body (default: 5)
  --never-returning-functions <members names>
                        Complete name of functions that never returns. When
                        checking for inconsistent-return-statements if a never
                        returning function is called then it will be
                        considered as an explicit return statement and no
                        message will be printed. (default: ('sys.exit',
                        'argparse.parse_error'))

Basic:
  --good-names <names>  Good variable names which should always be accepted,
                        separated by a comma. (default: ('i', 'j', 'k', 'ex',
                        'Run', '_'))
  --good-names-rgxs <names>
                        Good variable names 

                        consideration. (default: )

Imports:
  BaseChecker for import statements.

  --deprecated-modules <modules>
                        Deprecated modules which should not be used, separated
                        by a comma. (default: ())
  --preferred-modules <module:preferred-module>
                        Couples of modules and preferred modules, separated by
                        a comma. (default: ())
  --import-graph <file.gv>
                        Output a graph (.gv or any supported image format) of
                        all (i.e. internal and external) dependencies to the
                        given file (report RP0402 must not be disabled).
                        (default: )
  --ext-import-graph <file.gv>
                        Output a graph (.gv or any supported image format) of
                        external dependencies to the given file (report RP0402
                        must not be disabled). (default: )
  --int-import-graph <file.

                        one word per line. (default: )
  --spelling-store-unknown-words <y or n>
                        Tells whether to store unknown words to the private
                        dictionary (see the --spelling-private-dict-file
                        option) instead of raising a message. (default: n)
  --max-spelling-suggestions N
                        Limits count of emitted suggestions for spelling
                        mistakes. (default: 4)
  --spelling-ignore-comment-directives <comma separated words>
                        List of comma separated words that should be
                        considered directives if they appear at the beginning
                        of a comment and should not be checked. (default: fmt:
                        on,fmt: off,noqa:,noqa,nosec,isort:skip,mypy:)

Similarities:
  Checks for similarities and duplicated code.

  --min-similarity-lines <int>
                        Minimum lines number of a similarity. (default: 4)